In [3]:
import pandas as pd
import random



# Viz
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [9]:
# Create a DataFrame for players 1 to 16 with a column "alive" set to 1
data = {'player': [i for i in range(1, 17)], 'alive': [1] * 16}
df = pd.DataFrame(data)

In [4]:
list(df.alive)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:
df.iloc[0].alive

1

In [11]:
def run_games(players, num_steps):
    players_copy = players.copy()
    lead_player = 0
    for i in range(num_steps - 1):
        random_num = random.choice([0, 1])
        if random_num == 1:
            continue
        else:
            if lead_player < len(players_copy):
                # Lead Player died
                players_copy.at[lead_player, 'alive'] = 0
                lead_player += 1
    return list(players_copy.alive)

In [12]:
run_games(df, 20)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [13]:
num_trials = 10000

In [19]:
results = []
for _ in range(num_trials):
    result = run_games(df, 20)
    results.append(result)

In [20]:
df_results = pd.DataFrame(results)

In [34]:
alive_counts = df_results.sum(axis=1)

In [35]:
alive_counts

0        6
1        7
2        6
3        9
4        6
        ..
9995     8
9996     9
9997     8
9998     7
9999    10
Length: 10000, dtype: int64

In [11]:
df_results.rename(columns = {'0': 'num_of_survivals'}, inplace=True)

In [12]:
df_results.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .set_caption("Stats Summary of Numeric Variables")\
                            .background_gradient(subset=['min'], cmap='Reds')\
                            .background_gradient(subset=['max'], cmap='Greens')\
                            .background_gradient(subset=['std'], cmap='GnBu')\
                            .background_gradient(subset=['50%'], cmap='GnBu')

,count,mean,std,min,25%,50%,75%,max
0,10000.000000,6.473300,2.185764,0.000000,5.000000,6.000000,8.000000,15.000000


In [37]:
alive_counts_sorted = alive_counts.sort_values()

In [42]:
palletes = {
   'continuos':{'blues': ['#03045E', '#023E8A', '#0077B6', '#0077B6', '#0096C7', '#00B4D8', '#48CAE4', '#90E0EF', '#ADE8F4', '#CAF0F8'],
                'green_n_blues': ['#D9ED92', '#B5E48C', '#99D98C', '#76C893', '#52B69A', '#34A0A4', '#168AAD', '#1A759F', '#1E6091', '#184E77']
               }
            }

In [55]:
fig = px.histogram(alive_counts_sorted, x=alive_counts_sorted,
                    marginal='box',
                    opacity=0.55,
                    facet_col_spacing=0.05,
                    color_discrete_sequence=palletes['continuos']['blues'])


# Update layout
fig.update_layout(
    template='plotly_white',  # Example template, you can replace with your specific template
    title={
        'text': '<sup>What is your likely surviving the Bridge Game in Squid Game?</sup>', 
        'x': 0.5

    }, 
    yaxis_title='Total Frequencies', 
    xaxis_title=' ',
    bargap=0.7
    )


# Add annotations
fig.add_annotation(
    x=0.5, y=-0.125, 
    align='center', 
    font=dict(size=12),
    textangle=0, 
    xref="paper", 
    yref="paper", 
    showarrow=False,
    text="<span style='font-size:16px;'><Total survivors>"
)

# Show the plot
fig.show()

Calculating the survivate probability of each players. Which Number Should I choose to maximize the survival probability

In [58]:
survival_counts = pd.DataFrame(0, index=range(num_trials), columns=df['player'])

In [59]:
survival_counts

player,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
for i in range(num_trials):
    survival_counts.iloc[i] = run_games(df, 20)

In [66]:
survival_probabilities = survival_counts.sum() / num_trials

In [71]:
survival_rates = survival_probabilities.values

# Create the scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=survival_probabilities.index,
    y=survival_rates,
    mode='markers',
    marker=dict(color='black', size=5)
))

# Update layout
fig.update_layout(
    title='Probability of survival based on player order number',
    xaxis_title='Player order number',
    yaxis_title='Probability',
    yaxis_tickformat='.0%',
    template='simple_white',
    height = 600
)

# Show the plot
fig.show()